In [ ]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from link_prediction import *

In [2]:
#Load the entity embeddings dict
with open("dbpedia50_distmult_entity_embeddings.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("dbpedia50_distmult_predicate_embeddings.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

train_triples = pd.read_csv('dbpedia50_train.csv', dtype=str)
test_triples = pd.read_csv('dbpedia50_test_filtered.csv', dtype=str)

**Create Negative Samples**

In [3]:
training_object=Training(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='DistMult')
tail_df = training_object.create_training_data_filtered(n=1, creating_for="tail")
head_df = training_object.create_training_data_filtered(n=1, creating_for="head")
print(len(tail_df))
print(len(head_df))

Processing triples: 100%|██████████| 32388/32388 [00:24<00:00, 1331.13triple/s]

29346
11846


**Prepare Training Data**

In [4]:
train_triples['label']=1
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: training_object.get_embedding(row).cpu().numpy(), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)
train_df['label'].value_counts()

,count
label,
0,40004
1,32203


**Set Models**

**MLP**

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [6]:
param_grid_mlp = {
    'hidden_layer_sizes': [ (64, 64), (128, 128), (256, 256)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [25, 50, 100]
}


results_mlp = []
for hidden_layer_sizes in param_grid_mlp['hidden_layer_sizes']:
    for alpha in param_grid_mlp['alpha']:
        for learning_rate_init in param_grid_mlp['learning_rate_init']:
            for max_iter in param_grid_mlp['max_iter']:
                mlp = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    alpha=alpha,
                    learning_rate_init=learning_rate_init,
                    max_iter=max_iter,
                    solver='adam',
                    early_stopping=True,
                    batch_size=32,
                    random_state=42
                )
                mlp.fit(X_train, y_train)
                preds = mlp.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_mlp.append({
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'alpha': alpha,
                    'learning_rate_init': learning_rate_init,
                    'max_iter': max_iter,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_mlp = pd.DataFrame(results_mlp)

best_mlp_params = df_results_mlp.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for MLP:", best_mlp_params)

Best parameters for MLP: {'hidden_layer_sizes': (256, 256), 'alpha': 0.001, 'learning_rate_init': 0.001, 'max_iter': 100, 'f1_score': 0.6327856324035155, 'accuracy': 0.7338318792411024, 'precision_0': 0.6996351065872863, 'precision_1': 0.8222442899702085, 'recall_0': 0.9105223694076481, 'recall_1': 0.5142857142857142}


**XGB**

In [7]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    # 'scale_pos_weight': [0.5, 1, 2],  # commented out as provided
    'learning_rate': [0.01, 0.1, 0.2]
}


results_xgb = []
for n_estimators in param_grid_xgb['n_estimators']:
    for max_depth in param_grid_xgb['max_depth']:
        for learning_rate in param_grid_xgb['learning_rate']:
            xgb = XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                random_state=42,
            )
            xgb.fit(X_train, y_train)
            preds = xgb.predict(X_test)

            # Calculate metrics
            score_f1 = f1_score(y_test, preds)
            acc = accuracy_score(y_test, preds)
            precisions = precision_score(y_test, preds, average=None, zero_division=0)
            recalls = recall_score(y_test, preds, average=None, zero_division=0)

            results_xgb.append({
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'f1_score': score_f1,
                'accuracy': acc,
                'precision_0': precisions[0],
                'precision_1': precisions[1],
                'recall_0': recalls[0],
                'recall_1': recalls[1]
            })

df_results_xgb = pd.DataFrame(results_xgb)
best_xgb_params = df_results_xgb.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for XGBClassifier:", best_xgb_params)

Best parameters for XGBClassifier: {'n_estimators': 300.0, 'max_depth': 5.0, 'learning_rate': 0.2, 'f1_score': 0.8886138613861386, 'accuracy': 0.9002908184461986, 'precision_0': 0.9124968569273322, 'precision_1': 0.8853267570900123, 'recall_0': 0.9070232441889527, 'recall_1': 0.8919254658385093}


**LGBM**

In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2],
    'is_unbalance': [True, False]
}

results_lgbm = []
for n_estimators in param_grid_lgbm['n_estimators']:
    for max_depth in param_grid_lgbm['max_depth']:
        for learning_rate in param_grid_lgbm['learning_rate']:
            for is_unbalance in param_grid_lgbm['is_unbalance']:
                lgbm = LGBMClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    is_unbalance=is_unbalance,
                    random_state=42
                )
                lgbm.fit(X_train, y_train)
                preds = lgbm.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_lgbm.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'is_unbalance': is_unbalance,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_lgbm = pd.DataFrame(results_lgbm)
best_lgbm_params = df_results_lgbm.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for LGBM:", best_lgbm_params)

**Train Ensemble**

In [ ]:
# Best MLPClassifier
best_mlp = MLPClassifier(
    hidden_layer_sizes=best_mlp_params['hidden_layer_sizes'],
    alpha=best_mlp_params['alpha'],
    learning_rate_init=best_mlp_params['learning_rate_init'],
    max_iter=best_mlp_params['max_iter'],
    solver='adam',
    early_stopping=True,
    batch_size=32,
    random_state=42
)
best_mlp.fit(X_train, y_train)
preds_mlp = best_mlp.predict(X_test)
report_mlp = classification_report(y_test, preds_mlp)

# Best XGBClassifier
best_xgb = XGBClassifier(
    n_estimators=int(best_xgb_params['n_estimators']),
    max_depth=int(best_xgb_params['max_depth']),
    learning_rate=float(best_xgb_params['learning_rate']),
    random_state=42,
)
best_xgb.fit(X_train, y_train)
preds_xgb = best_xgb.predict(X_test)
report_xgb = classification_report(y_test, preds_xgb)

# Best LGBMClassifier
best_lgbm = LGBMClassifier(
    n_estimators=best_lgbm_params['n_estimators'],
    max_depth=best_lgbm_params['max_depth'],
    learning_rate=best_lgbm_params['learning_rate'],
    is_unbalance=best_lgbm_params['is_unbalance'],
    random_state=42
)
best_lgbm.fit(X_train, y_train)
preds_lgbm = best_lgbm.predict(X_test)
report_lgbm = classification_report(y_test, preds_lgbm)

######################################
# Ensemble the best models using VotingClassifier
######################################
ensemble_model = VotingClassifier(
    estimators=[
        ('mlp', best_mlp),
        ('xgb', best_xgb),
        ('lgbm', best_lgbm)
    ],
    voting='soft',  # Soft voting uses predicted probabilities
    n_jobs=-1
)

start_time = time.time()
ensemble_model.fit(X_train, y_train)
end_time = time.time()

ensemble_time = end_time - start_time
preds_ensemble = ensemble_model.predict(X_test)
ensemble_acc = accuracy_score(y_test, preds_ensemble)
report_ensemble = classification_report(y_test, preds_ensemble)

In [10]:
y_pred_ensemble = ensemble_model.predict(X_test)
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Ensemble Classification Report:\n", classification_report(y_test, y_pred_ensemble))

Ensemble Accuracy: 0.8976596039329733
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.90      0.91      4001
           1       0.88      0.89      0.89      3220

    accuracy                           0.90      7221
   macro avg       0.90      0.90      0.90      7221
weighted avg       0.90      0.90      0.90      7221



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
with open("/content/drive/MyDrive/link_prediction/tuning/dbpedia50_distmult_tuning.txt", "w") as f:
    f.write("##############################\n")
    f.write("MLPClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_mlp.to_string())
    f.write("\n\nBest Parameters for MLPClassifier:\n")
    f.write(str(best_mlp_params))
    f.write("\n\nClassification Report for Best MLPClassifier:\n")
    f.write(report_mlp)

    f.write("\n\n\n##############################\n")
    f.write("XGBClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_xgb.to_string())
    f.write("\n\nBest Parameters for XGBClassifier:\n")
    f.write(str(best_xgb_params))
    f.write("\n\nClassification Report for Best XGBClassifier:\n")
    f.write(report_xgb)

    f.write("\n\n\n##############################\n")
    f.write("LGBMClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_lgbm.to_string())
    f.write("\n\nBest Parameters for LGBMClassifier:\n")
    f.write(str(best_lgbm_params))
    f.write("\n\nClassification Report for Best LGBMClassifier:\n")
    f.write(report_lgbm)

    f.write("\n\n\n##############################\n")
    f.write("Ensemble Model Performance\n")
    f.write("##############################\n")
    f.write("Ensemble Accuracy: {:.4f}\n".format(ensemble_acc))
    f.write("Ensemble Classification Report:\n")
    f.write(report_ensemble)

print("Results saved to model_results.txt")

Results saved to model_results.txt


**Rerank Hitk**

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='DistMult', k=50)

Precomputing top-k predictions: 100%|██████████| 2098/2098 [00:12<00:00, 170.39it/s]


In [14]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10,20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19,20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 32.19% | tail: 41.08% | overall: 37.58%
k=1 | th=2 | head: 22.95% | tail: 35.23% | overall: 30.40%
k=1 | th=3 | head: 17.73% | tail: 31.27% | overall: 25.95%
k=1 | th=4 | head: 14.31% | tail: 28.61% | overall: 22.98%
k=1 | th=5 | head: 12.44% | tail: 26.68% | overall: 21.08%
k=1 | th=6 | head: 10.36% | tail: 24.99% | overall: 19.23%
k=1 | th=7 | head: 9.02% | tail: 23.97% | overall: 18.09%
k=1 | th=8 | head: 7.60% | tail: 23.15% | overall: 17.03%
k=1 | th=9 | head: 6.78% | tail: 22.38% | overall: 16.24%
k=1 | th=10 | head: 6.26% | tail: 22.18% | overall: 15.92%
k=1 | th=20 | head: 3.73% | tail: 19.72% | overall: 13.43%
k=1 | th=30 | head: 3.20% | tail: 18.51% | overall: 12.49%
k=1 | th=40 | head: 2.53% | tail: 17.88% | overall: 11.84%
k=1 | th=50 | head: 2.31% | tail: 17.45% | overall: 11.49%
k=3 | th=3 | head: 42.62% | tail: 51.28% | overall: 47.87%
k=3 | th=4 | head: 36.36% | tail: 47.46% | overall: 43.10%
k=3 | th=5 | head: 31.89% | tail: 43.35% | overall: 38.84%


In [15]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 32.19% | tail: 41.08% | overall: 37.58%
k=1 | th=2 | head: 30.25% | tail: 42.34% | overall: 37.58%
k=1 | th=3 | head: 29.96% | tail: 42.68% | overall: 37.67%
k=1 | th=4 | head: 30.10% | tail: 42.53% | overall: 37.64%
k=1 | th=5 | head: 30.25% | tail: 42.29% | overall: 37.55%
k=1 | th=6 | head: 29.96% | tail: 42.24% | overall: 37.41%
k=1 | th=7 | head: 29.88% | tail: 41.95% | overall: 37.20%
k=1 | th=8 | head: 29.88% | tail: 42.00% | overall: 37.23%
k=1 | th=9 | head: 29.88% | tail: 42.05% | overall: 37.26%
k=1 | th=10 | head: 29.96% | tail: 42.10% | overall: 37.32%
k=1 | th=20 | head: 29.88% | tail: 41.71% | overall: 37.06%
k=1 | th=30 | head: 29.81% | tail: 41.66% | overall: 37.00%
k=1 | th=40 | head: 29.73% | tail: 41.66% | overall: 36.97%
k=1 | th=50 | head: 29.51% | tail: 41.71% | overall: 36.91%
k=3 | th=3 | head: 42.62% | tail: 51.28% | overall: 47.87%
k=3 | th=4 | head: 42.85% | tail: 51.09% | overall: 47.85%
k=3 | th=5 | head: 42.92% | tail: 51.09% | overall:

In [16]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 32.19% | tail: 41.08% | overall: 37.58%
k=1 | th=2 | head: 31.52% | tail: 42.10% | overall: 37.94%
k=1 | th=3 | head: 31.15% | tail: 42.05% | overall: 37.76%
k=1 | th=4 | head: 31.07% | tail: 41.76% | overall: 37.55%
k=1 | th=5 | head: 31.22% | tail: 41.57% | overall: 37.50%
k=1 | th=6 | head: 30.92% | tail: 41.42% | overall: 37.29%
k=1 | th=7 | head: 30.55% | tail: 41.47% | overall: 37.17%
k=1 | th=8 | head: 30.25% | tail: 41.47% | overall: 37.06%
k=1 | th=9 | head: 30.33% | tail: 41.37% | overall: 37.03%
k=1 | th=10 | head: 29.96% | tail: 41.47% | overall: 36.94%
k=1 | th=20 | head: 29.81% | tail: 41.76% | overall: 37.06%
k=1 | th=30 | head: 29.73% | tail: 41.61% | overall: 36.94%
k=1 | th=40 | head: 29.66% | tail: 41.66% | overall: 36.94%
k=1 | th=50 | head: 29.36% | tail: 41.47% | overall: 36.70%
k=3 | th=3 | head: 42.62% | tail: 51.28% | overall: 47.87%
k=3 | th=4 | head: 42.85% | tail: 50.85% | overall: 47.70%
k=3 | th=5 | head: 43.22% | tail: 50.65% | overall:

In [17]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 32.19% | tail: 41.08% | overall: 37.58%
k=1 | th=2 | head: 31.37% | tail: 42.73% | overall: 38.26%
k=1 | th=3 | head: 31.07% | tail: 43.02% | overall: 38.32%
k=1 | th=4 | head: 31.45% | tail: 42.92% | overall: 38.41%
k=1 | th=5 | head: 31.45% | tail: 42.73% | overall: 38.29%
k=1 | th=6 | head: 31.22% | tail: 42.53% | overall: 38.08%
k=1 | th=7 | head: 31.15% | tail: 42.29% | overall: 37.91%
k=1 | th=8 | head: 31.00% | tail: 42.39% | overall: 37.91%
k=1 | th=9 | head: 31.22% | tail: 42.34% | overall: 37.97%
k=1 | th=10 | head: 31.07% | tail: 42.44% | overall: 37.97%
k=1 | th=20 | head: 30.85% | tail: 42.44% | overall: 37.88%
k=1 | th=30 | head: 30.77% | tail: 42.44% | overall: 37.85%
k=1 | th=40 | head: 30.85% | tail: 42.63% | overall: 37.99%
k=1 | th=50 | head: 30.85% | tail: 42.53% | overall: 37.94%
k=3 | th=3 | head: 42.62% | tail: 51.28% | overall: 47.87%
k=3 | th=4 | head: 43.00% | tail: 50.89% | overall: 47.79%
k=3 | th=5 | head: 43.22% | tail: 50.85% | overall: